# From an RDF KG to an entity matcher

The goal here is to automatically construct an Entity Matcher based on an RDF Knowledge Graph (KG). 
For the purpose of this demo we have constructed a sample knowledge graph of the pizzas proposed by the restaurant Le bisou, Rouen, France: <https://www.bisourouen.fr/>

The entity matcher used in the demo is effectively a spaCy span ruler. It means that the constructed matcher is rule-based and solely focused on linking the KG entities in a text by matching the raw strings.

The input KG is expressed in RDF. For simplicity we only focus on english.

In [ ]:
from pathlib import Path

import spacy

from buzz_el.graph import RDFGraphLoader
from buzz_el.entity_matcher import EntityMatcher

In [ ]:
# path to the KG RDF file
pizza_kg_filepath = Path("./data/pizzas_bisou_sample.ttl")

In [ ]:
# Example documents
reviews = [
    """Absolutely delightful! The Bianca Castafiore pizza with its ricotta cream base offers a perfect blend of flavors. 
    I love the options of goat cheese and vegetarian toppings—especially the combination of black pepper, goat cheese, 
    honey, fior di latte mozzarella, spinach, and walnuts. A true treat for the taste buds!""",
    """A flavor explosion! The BurraTadah pizza, with its tomato base and pork pizza category, is a must-try. 
    The combination of black pepper, cherry tomatoes, mozzarella di burrata, fior di latte mozzarella, olive oil, 
    Parma ham, Parmesan cheese, and rocket creates a symphony of deliciousness. Simply irresistible!""",
    """Fit for royalty! The God Save The King pizza, featuring a tomato base and exquisite pork toppings, is a culinary masterpiece. 
    The Parisian mushroom carpaccio, ham from Paris with herbs, fior di latte mozzarella, and olives create a regal flavor profile. 
    A royal feast for the senses!"""
]

In [ ]:
# create a minimum spaCy Language (i.e., a pipeline)
nlp = spacy.load(
        "en_core_web_sm",
        exclude=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer", "ner"], # we make the pipeline as small as possible for our little demo
    )

corpus = [doc for doc in nlp.pipe(reviews)]

The main goal of the graph loader is to construct an index of the entity URIs associated with their representative strings.

By default the graph loader will consider an entity anything that is the subject of a triple containing an labelling property. And it will consider a representative string the object of such triple.

You can provide you own annotation properties. If not a usual one (i.e., for which RDFlib has a namespace defined), you need to provide the full URI. By default it will use rdfs:label.

You can also filter using a language tag.

In [ ]:
# Build a graph loader
kg_loader = RDFGraphLoader(
    kg_file_path=pizza_kg_filepath,
    label_properties={"skos:altLabel", "rdfs:label", "skos:prefLabel"}, # define what are the annotation properties in RDF KG
    lang_filter_tag="en" # optionally define a language to focus on
)

pizza_kg = kg_loader.build_knowledge_graph()

In [ ]:
# build an entity matcher
entity_matcher = EntityMatcher(
    knowledge_graph=pizza_kg,
    spacy_model=nlp
)

In [ ]:
# process your documents
corpus = [doc for doc in entity_matcher.pipe(corpus)]

In [ ]:
# The spans are stored in the Doc.spans attribute by default in the "string" key.
corpus[0].spans

In [ ]:
for doc in corpus:
    print(doc)
    for span in doc.spans["string"]:
        print(span.text, span.start, span.end, span.id_.replace("http://www.msesboue.org/o/pizza-data-demo/bisou#", "pizza:"))
    print()